In [2]:
import pandas as pd

In [3]:
p = "mean of diseased pixels in red plane, mean of diseased pixels in green plane, mean of diseased pixels in blue plane, mean of background pixels in red plane, mean of background pixels in green plane, mean of background pixels in blue plane, mean of boundary region pixels of disease in red plane, mean of boundary region pixels of disease in green plane, mean of boundary region pixels of disease in blue plane, mean of core region pixels of disease in red plane, mean of core region pixels of disease in green plane, mean of core region pixels of disease in blue plane, standard deviation of diseased pixels in red plane, standard deviation of diseased pixels in green plane, standard deviation of diseased pixels in blue plane, standard deviation of background pixels in red plane, standard deviation of background pixels in green plane, standard deviation of background pixels in blue plane, standard deviation of boundary region pixels of disease in red plane, standard deviation of boundary region pixels of disease in green plane, standard deviation of boundary region pixels of disease in blue plane, standard deviation of core region pixels of disease in red plane, standard deviation of core region pixels of disease in green plane, standard deviation of core region pixels of disease in blue plane."

In [4]:
q = p.lower().split(",")

In [5]:
d = {'feature_of_data':q}
df = pd.DataFrame(d)

In [6]:
df

,feature_of_data
0,mean of diseased pixels in red plane
1,mean of diseased pixels in green plane
2,mean of diseased pixels in blue plane
3,mean of background pixels in red plane
4,mean of background pixels in green plane
5,mean of background pixels in blue plane
6,mean of boundary region pixels of disease in ...
7,mean of boundary region pixels of disease in ...
8,mean of boundary region pixels of disease in ...
9,mean of core region pixels of disease in red ...


In [7]:
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
PATH = os.getcwd()
# Define data path
data_path = PATH + '/Datasets'
data_dir_list = os.listdir(data_path)
print(data_dir_list)

['Bacterial leaf blight', 'Brown spot', 'Leaf smut']


In [26]:
import cv2
import numpy as np

def image_masking(img):
    BLUR = 21
    CANNY_THRESH_1 = 10
    CANNY_THRESH_2 = 200
    MASK_DILATE_ITER = 10
    MASK_ERODE_ITER = 10
    MASK_COLOR = (0.0,0.0,0.0) # In BGR format
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)
    contour_info = []
    contours, __ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    for c in contours:
        contour_info.append((c, cv2.isContourConvex(c), cv2.contourArea(c),))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    
    max_contour = contour_info[0]
    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))
    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
    
    mask_stack = np.dstack([mask]*3)
    mask_stack  = mask_stack.astype('float32') / 255.0
    img = img.astype('float32') / 255.0
    masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR)
    masked = (masked * 255).astype('uint8')
    resized_mask= cv2.resize(masked,(256,256),interpolation = cv2.INTER_AREA)
    #cv2.imshow("reshaped_img",resized_mask)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return resized_mask

#if __name__ == '__main__':
#    filepath = "DSC_0373.JPG"
#    image_masking(filepath)

In [27]:
num_classes = 3

labels_name={'Bacterial leaf blight':0,'Brown spot':1,'Leaf smut':2}

img_data_list=[]
labels_list = []

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loading the images of dataset-'+'{}\n'.format(dataset))
    label = labels_name[dataset]
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        img_data_list.append(input_img)
        labels_list.append(label)

img_data = np.array(img_data_list)
print (img_data.shape)

Loading the images of dataset-Bacterial leaf blight

Loading the images of dataset-Brown spot

Loading the images of dataset-Leaf smut

(120,)


In [28]:
reshaped_img = []
for i in range(120):
    reshaped_img.append(cv2.resize(img_data[i],(256,256)))

In [29]:
R1 = []
G1 = []
B1 = []
for i in range(120):
    B,G,R = cv2.split(reshaped_img[i])
    R1.append(R)
    G1.append(G)
    B1.append(B)

In [30]:
print(R1[119].shape)
print(G1[119].shape)
print(B1[119].shape)

(256, 256)
(256, 256)
(256, 256)


In [31]:
for i in range(120):
    print(i)
    image_masking(reshaped_img[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


In [117]:
#core mean
sp_b = []
sp_g = []
sp_r = []
for i in range(120):
    b,g,r  = cv2.split(image_masking(reshaped_img[i]))
    sp_b.append(b.mean())
    sp_g.append(g.mean())
    sp_r.append(r.mean())

In [119]:
sp_b[10]

7.340118408203125

In [118]:
#core std
std_sp_b = []
std_sp_g = []
std_sp_r = []
for i in range(120):
    b,g,r  = cv2.split(image_masking(reshaped_img[i]))
    std_sp_b.append(b.std())
    std_sp_g.append(g.std())
    std_sp_r.append(r.std())

In [120]:
std_sp_b[10]

16.581156006366502

In [39]:
print(reshaped_img[100])
print(sp_bgr[100])

[[[218 223 239]
  [219 225 236]
  [219 225 236]
  ...
  [217 223 236]
  [219 224 239]
  [218 222 240]]

 [[218 223 237]
  [218 224 237]
  [219 225 238]
  ...
  [216 221 237]
  [218 223 238]
  [216 223 239]]

 [[218 224 237]
  [218 224 237]
  [220 226 239]
  ...
  [215 223 237]
  [215 223 238]
  [215 222 239]]

 ...

 [[216 223 240]
  [217 224 239]
  [216 223 238]
  ...
  [216 222 236]
  [216 221 237]
  [216 220 238]]

 [[216 223 240]
  [216 223 238]
  [216 223 238]
  ...
  [218 223 238]
  [216 221 236]
  [217 221 239]]

 [[216 223 240]
  [216 223 238]
  [216 223 238]
  ...
  [216 224 237]
  [217 222 237]
  [217 223 240]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0

In [40]:
rgb = []
for i in range(120):
    rgb.append(cv2.cvtColor(reshaped_img[i],cv2.COLOR_BGR2RGB))
hsv = []
canny = []
for i in range(120):
    hsv.append(cv2.cvtColor(rgb[i],cv2.COLOR_RGB2HSV))
    canny.append(cv2.Canny(rgb[i],100,200))
for i in range(120):
    cv2.imshow("rgb_of_image",rgb[i])
    cv2.imshow("hsv_of_image",hsv[i])
    cv2.imshow("canny",canny[i])
    cv2.waitKey(0)
    

cv2.destroyAllWindows()

In [59]:
R1 = []
G1 = []
B1 = []
for i in range(120):
    R,G,B = cv2.split(rgb[i])
    R1.append(R)
    G1.append(G)
    B1.append(B)

In [60]:
canny_R = []
canny_G = []
canny_B = []
for i in range(120):
    canny_R.append(cv2.Canny(R1[100],100,200))
    canny_G.append(cv2.Canny(G1[100],100,200))
    canny_B.append(cv2.Canny(B1[100],100,200))

In [123]:
#boundary region pixel
mean_canny_R = []
mean_canny_G = []
mean_canny_B = []
for i in range(120):
    mean_canny_R.append(canny_R[i].mean())
    mean_canny_G.append(canny_G[i].mean())
    mean_canny_B.append(canny_B[i].mean())

In [124]:
#boundary region pixel
std_canny_R = []
std_canny_G = []
std_canny_B = []
for i in range(120):
    std_canny_R.append(canny_R[i].std())
    std_canny_G.append(canny_G[i].std())
    std_canny_B.append(canny_B[i].std())

In [126]:
print(mean_canny_B[10])
print(std_canny_B[10])

2.498016357421875
25.114818044744638


In [69]:
mean_desised_pixel_R = []
mean_desised_pixel_G = []
mean_desised_pixel_B = []
for i in range(120):
    mean_desised_pixel_R.append(R1[i].mean())
    mean_desised_pixel_G.append(G1[i].mean())
    mean_desised_pixel_B.append(B1[i].mean())

In [127]:
std_desised_pixel_R = []
std_desised_pixel_G = []
std_desised_pixel_B = []
for i in range(120):
    std_desised_pixel_R.append(R1[i].std())
    std_desised_pixel_G.append(G1[i].std())
    std_desised_pixel_B.append(B1[i].std())

In [128]:
print(mean_desised_pixel_B[30])
print(std_desised_pixel_B[30])

163.30738830566406
83.37498944582046


In [71]:
mean_desised_pixel_B

[163.721435546875,
 159.2592315673828,
 154.84542846679688,
 131.14024353027344,
 143.45098876953125,
 164.451904296875,
 122.63597106933594,
 156.82635498046875,
 163.77574157714844,
 145.7988739013672,
 142.19650268554688,
 144.50631713867188,
 148.72628784179688,
 168.267578125,
 157.45455932617188,
 151.8960418701172,
 177.8815155029297,
 129.88404846191406,
 156.5992889404297,
 132.83895874023438,
 146.18113708496094,
 117.60383605957031,
 129.4631805419922,
 140.88858032226562,
 137.1263427734375,
 116.99295043945312,
 160.9743194580078,
 180.1693115234375,
 144.88316345214844,
 136.82627868652344,
 163.30738830566406,
 138.09024047851562,
 139.7640380859375,
 139.03024291992188,
 140.616455078125,
 160.67811584472656,
 203.0330810546875,
 226.3924560546875,
 181.9353485107422,
 164.9491729736328,
 151.44302368164062,
 124.73832702636719,
 117.49169921875,
 106.86454772949219,
 105.50112915039062,
 133.69699096679688,
 79.37181091308594,
 93.10633850097656,
 35.978302001953125,
 

In [67]:
mean_canny_R

[2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736328125,
 2.0038604736

In [63]:
cv2.imshow("canny_R",canny_R[50])
cv2.imshow("canny_G",canny_G[50])
cv2.imshow("canny_B",canny_B[70])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [115]:
back_gr_r = []
back_gr_g = []
back_gr_b = []
for i in range(120):
    ret,thresh_r=cv2.threshold(R1[i],50,255,cv2.THRESH_BINARY)
    ret,thresh_g=cv2.threshold(G1[i],50,255,cv2.THRESH_BINARY)
    ret,thresh_b=cv2.threshold(B1[i],50,255,cv2.THRESH_BINARY)
    back_gr_r.append(thresh_r.mean())
    back_gr_g.append(thresh_g.mean())
    back_gr_b.append(thresh_b.mean())

In [121]:
std_back_gr_r = []
std_back_gr_g = []
std_back_gr_b = []
for i in range(120):
    ret,thresh_r=cv2.threshold(R1[i],50,255,cv2.THRESH_BINARY)
    ret,thresh_g=cv2.threshold(G1[i],50,255,cv2.THRESH_BINARY)
    ret,thresh_b=cv2.threshold(B1[i],50,255,cv2.THRESH_BINARY)
    std_back_gr_r.append(thresh_r.std())
    std_back_gr_g.append(thresh_g.std())
    std_back_gr_b.append(thresh_b.std())

In [122]:
print(std_back_gr_b[10])
print(back_gr_b[10])

115.22202767386727
182.09060668945312


In [116]:
back_gr_b

[201.21482849121094,
 195.4639434814453,
 188.33175659179688,
 158.67462158203125,
 181.01280212402344,
 203.84902954101562,
 158.17657470703125,
 191.35116577148438,
 201.86073303222656,
 173.58489990234375,
 182.09060668945312,
 178.16070556640625,
 169.76783752441406,
 211.64268493652344,
 187.2344970703125,
 185.0749969482422,
 214.1484832763672,
 152.32452392578125,
 190.4718017578125,
 164.00527954101562,
 181.5886688232422,
 134.83840942382812,
 152.2039031982422,
 171.91177368164062,
 161.89247131347656,
 141.51145935058594,
 194.14100646972656,
 219.63478088378906,
 164.65896606445312,
 160.0714874267578,
 197.69737243652344,
 171.26197814941406,
 175.71327209472656,
 166.18812561035156,
 173.44093322753906,
 200.92689514160156,
 232.05093383789062,
 237.3504638671875,
 223.84872436523438,
 200.35491943359375,
 169.11026000976562,
 133.9434814453125,
 149.06387329101562,
 209.81781005859375,
 205.12916564941406,
 154.54627990722656,
 77.48908996582031,
 112.5897216796875,
 62.

In [108]:
ret,thresh=cv2.threshold(B1[10],100,255,cv2.THRESH_BINARY)
cv2.imshow("ori",B1[10])
cv2.imshow("thresh",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#std
